In [1]:
import os
import time
import json
import pprint
import random
import numpy as np
from tqdm import tqdm, trange
from collections import defaultdict

In [2]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader



In [3]:


from cg_detr.config import BaseOptions
from cg_detr.start_end_dataset import \
    StartEndDataset, start_end_collate, prepare_batch_inputs
from cg_detr.inference import eval_epoch, start_inference, setup_model
from utils.basic_utils import AverageMeter, dict_to_markdown
from utils.model_utils import count_parameters

In [4]:
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s.%(msecs)03d:%(levelname)s:%(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
)


In [5]:
def set_seed(seed, use_cuda=True):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_cuda:
        torch.cuda.manual_seed_all(seed)

In [6]:
def set_seed(seed, use_cuda=True):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_cuda:
        torch.cuda.manual_seed_all(seed)


def train_epoch(model, criterion, train_loader, optimizer, opt, epoch_i, tb_writer):
    logger.info(f"[Epoch {epoch_i+1}]")
    model.train()
    criterion.train()

    # init meters
    time_meters = defaultdict(AverageMeter)
    loss_meters = defaultdict(AverageMeter)

    num_training_examples = len(train_loader)
    timer_dataloading = time.time()
    for batch_idx, batch in tqdm(
        enumerate(train_loader), desc="Training Iteration", total=num_training_examples
    ):
        time_meters["dataloading_time"].update(time.time() - timer_dataloading)

        timer_start = time.time()

        model_inputs, targets = prepare_batch_inputs(
            batch[1], opt.device, non_blocking=opt.pin_memory
        )

        time_meters["prepare_inputs_time"].update(time.time() - timer_start)
        timer_start = time.time()

        outputs = model(**model_inputs, targets=targets)
        loss_dict = criterion(outputs, targets)
        weight_dict = criterion.weight_dict
        losses = sum(
            loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict
        )
        time_meters["model_forward_time"].update(time.time() - timer_start)

        timer_start = time.time()
        optimizer.zero_grad()
        losses.backward()
        if opt.grad_clip > 0:
            nn.utils.clip_grad_norm_(model.parameters(), opt.grad_clip)
        optimizer.step()
        time_meters["model_backward_time"].update(time.time() - timer_start)

        loss_dict["loss_overall"] = float(losses)  # for logging only
        for k, v in loss_dict.items():
            loss_meters[k].update(
                float(v) * weight_dict[k] if k in weight_dict else float(v)
            )

        timer_dataloading = time.time()
        if opt.debug and batch_idx == 3:
            break

    # print/add logs
    tb_writer.add_scalar(
        "Train/lr", float(optimizer.param_groups[0]["lr"]), epoch_i + 1
    )
    for k, v in loss_meters.items():
        tb_writer.add_scalar("Train/{}".format(k), v.avg, epoch_i + 1)

    to_write = opt.train_log_txt_formatter.format(
        time_str=time.strftime("%Y_%m_%d_%H_%M_%S"),
        epoch=epoch_i + 1,
        loss_str=" ".join(
            ["{} {:.4f}".format(k, v.avg) for k, v in loss_meters.items()]
        ),
    )
    with open(opt.train_log_filepath, "a") as f:
        f.write(to_write)

    logger.info("Epoch time stats:")
    for name, meter in time_meters.items():
        d = {k: f"{getattr(meter, k):.4f}" for k in ["max", "min", "avg"]}
        logger.info(f"{name} ==> {d}")

In [7]:
from torch.utils.tensorboard import SummaryWriter

In [8]:
def train(model, criterion, optimizer, lr_scheduler, train_dataset, val_dataset, opt):
    if opt.device.type == "cuda":
        logger.info("CUDA enabled.")
        model.to(opt.device)

    tb_writer = SummaryWriter(opt.tensorboard_log_dir)
    tb_writer.add_text("hyperparameters", dict_to_markdown(vars(opt), max_str_len=None))
    opt.train_log_txt_formatter = "{time_str} [Epoch] {epoch:03d} [Loss] {loss_str}\n"
    opt.eval_log_txt_formatter = "{time_str} [Epoch] {epoch:03d} [Loss] {loss_str} [Metrics] {eval_metrics_str}\n"

    train_loader = DataLoader(
        train_dataset,
        collate_fn=start_end_collate,
        batch_size=opt.bsz,
        num_workers=opt.num_workers,
        shuffle=True,
        pin_memory=opt.pin_memory,
    )

    prev_best_score = 0.0
    es_cnt = 0
    # start_epoch = 0
    if opt.start_epoch is None:
        start_epoch = -1 if opt.eval_untrained else 0
    else:
        start_epoch = opt.start_epoch
    save_submission_filename = "latest_{}_{}_preds.jsonl".format(
        opt.dset_name, opt.eval_split_name
    )
    for epoch_i in trange(start_epoch, opt.n_epoch, desc="Epoch"):
        if epoch_i > -1:
            train_epoch(
                model, criterion, train_loader, optimizer, opt, epoch_i, tb_writer
            )
            lr_scheduler.step()
        eval_epoch_interval = opt.eval_epoch
        if opt.eval_path is not None and (epoch_i + 1) % eval_epoch_interval == 0:
            with torch.no_grad():
                metrics_no_nms, metrics_nms, eval_loss_meters, latest_file_paths = (
                    eval_epoch(
                        model,
                        val_dataset,
                        opt,
                        save_submission_filename,
                        epoch_i,
                        criterion,
                        tb_writer,
                    )
                )

            # log
            to_write = opt.eval_log_txt_formatter.format(
                time_str=time.strftime("%Y_%m_%d_%H_%M_%S"),
                epoch=epoch_i,
                loss_str=" ".join(
                    ["{} {:.4f}".format(k, v.avg) for k, v in eval_loss_meters.items()]
                ),
                eval_metrics_str=json.dumps(metrics_no_nms),
            )

            with open(opt.eval_log_filepath, "a") as f:
                f.write(to_write)
            logger.info(
                "metrics_no_nms {}".format(
                    pprint.pformat(metrics_no_nms["brief"], indent=4)
                )
            )
            if metrics_nms is not None:
                logger.info(
                    "metrics_nms {}".format(
                        pprint.pformat(metrics_nms["brief"], indent=4)
                    )
                )

            metrics = metrics_no_nms
            for k, v in metrics["brief"].items():
                tb_writer.add_scalar(f"Eval/{k}", float(v), epoch_i + 1)

            if opt.dset_name in ["hl"]:
                stop_score = metrics["brief"]["MR-full-mAP"]
            else:
                stop_score = (
                    metrics["brief"]["MR-full-R1@0.7"]
                    + metrics["brief"]["MR-full-R1@0.5"]
                ) / 2

            if stop_score > prev_best_score:
                es_cnt = 0
                prev_best_score = stop_score

                checkpoint = {
                    "model": model.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    "lr_scheduler": lr_scheduler.state_dict(),
                    "epoch": epoch_i,
                    "opt": opt,
                }
                torch.save(checkpoint, opt.ckpt_filepath.replace(".ckpt", "_best.ckpt"))

                best_file_paths = [
                    e.replace("latest", "best") for e in latest_file_paths
                ]
                for src, tgt in zip(latest_file_paths, best_file_paths):
                    os.renames(src, tgt)
                logger.info("The checkpoint file has been updated.")
            else:
                es_cnt += 1
                if opt.max_es_cnt != -1 and es_cnt > opt.max_es_cnt:  # early stop
                    with open(opt.train_log_filepath, "a") as f:
                        f.write(f"Early Stop at epoch {epoch_i}")
                    logger.info(
                        f"\n>>>>> Early stop at epoch {epoch_i}  {prev_best_score}\n"
                    )
                    break

            # save ckpt
            checkpoint = {
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "lr_scheduler": lr_scheduler.state_dict(),
                "epoch": epoch_i,
                "opt": opt,
            }
            torch.save(checkpoint, opt.ckpt_filepath.replace(".ckpt", "_latest.ckpt"))

        # save_interval = 10 if "subs_train" in opt.train_path else 50  # smaller for pretrain
        # if (epoch_i + 1) % save_interval == 0 or (epoch_i + 1) % opt.lr_drop == 0:  # additional copies
        #     checkpoint = {
        #         "model": model.state_dict(),
        #         "optimizer": optimizer.state_dict(),
        #         "epoch": epoch_i,
        #         "opt": opt
        #     }
        #     torch.save(checkpoint, opt.ckpt_filepath.replace(".ckpt", f"_e{epoch_i:04d}.ckpt"))

        if opt.debug:
            break

    tb_writer.close()

In [9]:
def train_hl(
    model, criterion, optimizer, lr_scheduler, train_dataset, val_dataset, opt
):
    if opt.device.type == "cuda":
        logger.info("CUDA enabled.")
        model.to(opt.device)

    tb_writer = SummaryWriter(opt.tensorboard_log_dir)
    tb_writer.add_text("hyperparameters", dict_to_markdown(vars(opt), max_str_len=None))
    opt.train_log_txt_formatter = "{time_str} [Epoch] {epoch:03d} [Loss] {loss_str}\n"
    opt.eval_log_txt_formatter = "{time_str} [Epoch] {epoch:03d} [Loss] {loss_str} [Metrics] {eval_metrics_str}\n"

    train_loader = DataLoader(
        train_dataset,
        collate_fn=start_end_collate,
        batch_size=opt.bsz,
        num_workers=opt.num_workers,
        shuffle=True,
        pin_memory=opt.pin_memory,
    )

    prev_best_score = 0.0
    es_cnt = 0
    # start_epoch = 0
    if opt.start_epoch is None:
        start_epoch = -1 if opt.eval_untrained else 0
    else:
        start_epoch = opt.start_epoch
    save_submission_filename = "latest_{}_{}_preds.jsonl".format(
        opt.dset_name, opt.eval_split_name
    )
    for epoch_i in trange(start_epoch, opt.n_epoch, desc="Epoch"):
        if epoch_i > -1:
            train_epoch(
                model, criterion, train_loader, optimizer, opt, epoch_i, tb_writer
            )
            lr_scheduler.step()
        eval_epoch_interval = 5
        if opt.eval_path is not None and (epoch_i + 1) % eval_epoch_interval == 0:
            with torch.no_grad():
                metrics_no_nms, metrics_nms, eval_loss_meters, latest_file_paths = (
                    eval_epoch(
                        model,
                        val_dataset,
                        opt,
                        save_submission_filename,
                        epoch_i,
                        criterion,
                        tb_writer,
                    )
                )

            # log
            to_write = opt.eval_log_txt_formatter.format(
                time_str=time.strftime("%Y_%m_%d_%H_%M_%S"),
                epoch=epoch_i,
                loss_str=" ".join(
                    ["{} {:.4f}".format(k, v.avg) for k, v in eval_loss_meters.items()]
                ),
                eval_metrics_str=json.dumps(metrics_no_nms),
            )

            with open(opt.eval_log_filepath, "a") as f:
                f.write(to_write)
            logger.info(
                "metrics_no_nms {}".format(
                    pprint.pformat(metrics_no_nms["brief"], indent=4)
                )
            )
            if metrics_nms is not None:
                logger.info(
                    "metrics_nms {}".format(
                        pprint.pformat(metrics_nms["brief"], indent=4)
                    )
                )

            metrics = metrics_no_nms
            for k, v in metrics["brief"].items():
                tb_writer.add_scalar(f"Eval/{k}", float(v), epoch_i + 1)

            # stop_score = metrics["brief"]["MR-full-mAP"]
            stop_score = metrics["brief"]["mAP"]
            if stop_score > prev_best_score:
                es_cnt = 0
                prev_best_score = stop_score

                checkpoint = {
                    "model": model.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    "lr_scheduler": lr_scheduler.state_dict(),
                    "epoch": epoch_i,
                    "opt": opt,
                }
                torch.save(checkpoint, opt.ckpt_filepath.replace(".ckpt", "_best.ckpt"))

                best_file_paths = [
                    e.replace("latest", "best") for e in latest_file_paths
                ]
                for src, tgt in zip(latest_file_paths, best_file_paths):
                    os.renames(src, tgt)
                logger.info("The checkpoint file has been updated.")
            else:
                es_cnt += 1
                if opt.max_es_cnt != -1 and es_cnt > opt.max_es_cnt:  # early stop
                    with open(opt.train_log_filepath, "a") as f:
                        f.write(f"Early Stop at epoch {epoch_i}")
                    logger.info(
                        f"\n>>>>> Early stop at epoch {epoch_i}  {prev_best_score}\n"
                    )
                    break

            # save ckpt
            checkpoint = {
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "lr_scheduler": lr_scheduler.state_dict(),
                "epoch": epoch_i,
                "opt": opt,
            }
            torch.save(checkpoint, opt.ckpt_filepath.replace(".ckpt", "_latest.ckpt"))

        save_interval = (
            10 if "subs_train" in opt.train_path else 50
        )  # smaller for pretrain
        if (epoch_i + 1) % save_interval == 0 or (
            epoch_i + 1
        ) % opt.lr_drop == 0:  # additional copies
            checkpoint = {
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "epoch": epoch_i,
                "opt": opt,
            }
            torch.save(
                checkpoint, opt.ckpt_filepath.replace(".ckpt", f"_e{epoch_i:04d}.ckpt")
            )

        if opt.debug:
            break

    tb_writer.close()

In [10]:
def start_training():
    logger.info("Setup config, data and model...")
    opt = BaseOptions().parse()
    set_seed(opt.seed)
    if opt.debug:  # keep the model run deterministically
        # 'cudnn.benchmark = True' enabled auto finding the best algorithm for a specific input/net config.
        # Enable this only when input size is fixed.
        cudnn.benchmark = False
        cudnn.deterministic = True

    dataset_config = dict(
        dset_name=opt.dset_name,
        data_path=opt.train_path,
        v_feat_dirs=opt.v_feat_dirs,
        q_feat_dir=opt.t_feat_dir,
        q_feat_type="last_hidden_state",
        max_q_l=opt.max_q_l,
        max_v_l=opt.max_v_l,
        ctx_mode=opt.ctx_mode,
        data_ratio=opt.data_ratio,
        normalize_v=not opt.no_norm_vfeat,
        normalize_t=not opt.no_norm_tfeat,
        clip_len=opt.clip_length,
        max_windows=opt.max_windows,
        span_loss_type=opt.span_loss_type,
        txt_drop_ratio=opt.txt_drop_ratio,
        dset_domain=opt.dset_domain,
    )
    dataset_config["data_path"] = opt.train_path
    train_dataset = StartEndDataset(**dataset_config)

    if opt.eval_path is not None:
        dataset_config["data_path"] = opt.eval_path
        dataset_config["txt_drop_ratio"] = 0
        dataset_config["q_feat_dir"] = opt.t_feat_dir.replace(
            "sub_features", "text_features"
        )  # for pretraining
        # dataset_config["load_labels"] = False  # uncomment to calculate eval loss

        eval_dataset = StartEndDataset(**dataset_config)

    else:
        eval_dataset = None

    model, criterion, optimizer, lr_scheduler = setup_model(opt)
    logger.info(f"Model {model}")
    count_parameters(model)
    logger.info("Start Training...")

    # For tvsum dataset, use train_hl function
    if opt.dset_name in ["tvsum", "youtube_uni"]:
        train_hl(
            model, criterion, optimizer, lr_scheduler, train_dataset, eval_dataset, opt
        )
    else:
        train(
            model, criterion, optimizer, lr_scheduler, train_dataset, eval_dataset, opt
        )

    return (
        opt.ckpt_filepath.replace(".ckpt", "_best.ckpt"),
        opt.eval_split_name,
        opt.eval_path,
        opt.debug,
        opt,
    )

In [11]:
if __name__ == '__main__':
    best_ckpt_path, eval_split_name, eval_path, debug = start_training()
    if not debug:
        input_args = ["--resume", best_ckpt_path,
                      "--eval_split_name", eval_split_name,
                      "--eval_path", eval_path]

        import sys
        sys.argv[1:] = input_args
        logger.info("\n\n\nFINISHED TRAINING!!!")
        logger.info("Evaluating model at {}".format(best_ckpt_path))
        logger.info("Input args {}".format(sys.argv[1:]))
        start_inference()

2025-04-16 09:56:57.393:INFO:__main__ - Setup config, data and model...
usage: ipykernel_launcher.py [-h]
                             [--dset_name {hl,tvsum,charadesSTA,tacos,nlq,youtube_uni}]
                             [--dset_domain DSET_DOMAIN]
                             [--eval_split_name EVAL_SPLIT_NAME] [--debug]
                             [--data_ratio DATA_RATIO]
                             [--results_root RESULTS_ROOT] [--exp_id EXP_ID]
                             [--seed SEED] [--device DEVICE]
                             [--num_workers NUM_WORKERS] [--no_pin_memory]
                             [--lr LR] [--lr_drop LR_DROP] [--wd WD]
                             [--n_epoch N_EPOCH] [--max_es_cnt MAX_ES_CNT]
                             [--bsz BSZ] [--eval_bsz EVAL_BSZ]
                             [--eval_epoch EVAL_EPOCH] [--grad_clip GRAD_CLIP]
                             [--eval_untrained] [--resume RESUME]
                             [--resume_all] [--start_e

SystemExit: 2

f:\CG-Detr\CGDETR\venv\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [15]:
from run_on_video.data_utils import ClipFeatureExtractor
from run_on_video.model_utils import build_inference_model
from utils.tensor_utils import pad_sequences_1d
from cg_detr.span_utils import span_cxw_to_xx
from utils.basic_utils import l2_normalize_np_array
import torch.nn.functional as F
import numpy as np
import subprocess
from torch.utils.tensorboard import SummaryWriter

import os
import json
from pytube import YouTube
from moviepy.video.io.VideoFileClip import VideoFileClip

In [16]:
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(l.strip("\n")) for l in f.readlines()]

In [18]:
class CGDETRPredictor:
    def __init__(self, ckpt_path, clip_model_name_or_path="ViT-B/32", device="cuda"):
        self.clip_len = 2  # seconds
        self.device = device
        print("Loading feature extractors...")
        self.feature_extractor = ClipFeatureExtractor(
            framerate=1/self.clip_len, size=224, centercrop=True,
            model_name_or_path=clip_model_name_or_path, device=device
        )
        print("Loading trained CG-DETR model...")
        self.model = build_inference_model(ckpt_path).to(self.device)

    @torch.no_grad()
    def localize_moment(self, video_path, query_list):
        """
        Args:
            video_path: str, path to the video file
            query_list: List[str], each str is a query for this video
        """
        # construct model inputs
        n_query = len(query_list)
        video_feats = self.feature_extractor.encode_video(video_path)
        video_feats = F.normalize(video_feats, dim=-1, eps=1e-5)
        n_frames = len(video_feats)
        # add tef
        tef_st = torch.arange(0, n_frames, 1.0) / n_frames
        tef_ed = tef_st + 1.0 / n_frames
        tef = torch.stack([tef_st, tef_ed], dim=1).to(self.device)  # (n_frames, 2)
        video_feats = torch.cat([video_feats, tef], dim=1)
        assert n_frames <= 75, "The positional embedding of this pretrained CGDETR only support video up " \
                               "to 150 secs (i.e., 75 2-sec clips) in length"
        video_feats = video_feats.unsqueeze(0).repeat(n_query, 1, 1)  # (#text, T, d)
        video_mask = torch.ones(n_query, n_frames).to(self.device)
        query_feats = self.feature_extractor.encode_text(query_list)  # #text * (L, d)
        query_feats, query_mask = pad_sequences_1d(
            query_feats, dtype=torch.float32, device=self.device, fixed_length=None)
        query_feats = F.normalize(query_feats, dim=-1, eps=1e-5)
        model_inputs = dict(
            src_vid=video_feats,
            src_vid_mask=video_mask,
            src_txt=query_feats,
            src_txt_mask=query_mask,
            vid=None,
            qid=None
        )

        # decode outputs
        outputs = self.model(**model_inputs)
        # #moment_queries refers to the positional embeddings in CGDETR's decoder, not the input text query
        prob = F.softmax(outputs["pred_logits"], -1)  # (batch_size, #moment_queries=10, #classes=2)
        scores = prob[..., 0]  # * (batch_size, #moment_queries)  foreground label is 0, we directly take it
        pred_spans = outputs["pred_spans"]  # (bsz, #moment_queries, 2)
        _saliency_scores = outputs["saliency_scores"].half()  # (bsz, L)
        saliency_scores = []
        valid_vid_lengths = model_inputs["src_vid_mask"].sum(1).cpu().tolist()
        for j in range(len(valid_vid_lengths)):
            _score = _saliency_scores[j, :int(valid_vid_lengths[j])].tolist()
            _score = [round(e, 4) for e in _score]
            saliency_scores.append(_score)

        # compose predictions
        predictions = []
        video_duration = n_frames * self.clip_len
        for idx, (spans, score) in enumerate(zip(pred_spans.cpu(), scores.cpu())):
            spans = span_cxw_to_xx(spans) * video_duration
            # # (#queries, 3), [st(float), ed(float), score(float)]
            cur_ranked_preds = torch.cat([spans, score[:, None]], dim=1).tolist()
            cur_ranked_preds = sorted(cur_ranked_preds, key=lambda x: x[2], reverse=True)
            cur_ranked_preds = [[float(f"{e:.4f}") for e in row] for row in cur_ranked_preds]
            cur_query_pred = dict(
                query=query_list[idx],  # str
                vid=video_path,
                pred_relevant_windows=cur_ranked_preds,  # List([st(float), ed(float), score(float)])
                pred_saliency_scores=saliency_scores[idx]  # List(float), len==n_frames, scores for each frame
            )
            predictions.append(cur_query_pred)

        return predictions

In [19]:
def run_example():

    '''
    1) If you want to use the custom data, leave the url empty
    '''
    youtube_url = ''
    vid_st_sec, vid_ed_sec = 0.0, 0.0
    desired_query = ''

    '''
    2) If you want to run with a video from youtube, please enter the youtube_url, [st, ed] in seconds, and custom query
    # Maximum duration is 150 secs or lower. Recommend to use less than 150 secs.
    youtube_url = 'https://www.youtube.com/watch?v=geklhsKfw7I'
    vid_st_sec, vid_ed_sec = 60.0, 205.0
    desired_query = 'Girls having fun out side shop'
    '''
    # youtube_url = 'https://www.youtube.com/watch?v=geklhsKfw7I'
    # vid_st_sec, vid_ed_sec = 60.0, 205.0
    # desired_query = 'Girls having fun out side shop'


    # load example data
    from utils.basic_utils import load_jsonl

    if youtube_url != '':
        # vid = info['vid'] # "vid": "NUsG9BgSes0_210.0_360.0"
        queries = []
        queries.append({})
        file_name = youtube_url.split('/')[-1][8:] + '_' + str(vid_st_sec) + '_' + str(vid_ed_sec) + '.mp4'
        if os.path.exists(os.path.join('run_on_video/example', file_name)):
            video_path = os.path.join('run_on_video/example', file_name)
            queries[0]['query'] = desired_query
        else:
            try:
                yt = YouTube(youtube_url)
                stream = yt.streams.get_highest_resolution()
                video_path = os.path.join('./run_on_video/example', file_name)
                stream.download(output_path='./run_on_video/example', filename=file_name)
            except:
                print('Error downloading video')
                exit(1)



        with VideoFileClip(video_path) as video:
            new = video.subclip(vid_st_sec, vid_ed_sec)
            new.write_videofile(video_path, audio_codec='aac')


        queries[0]['query'] = 'A woman is talking to a camera.'
    else:
        video_path = "run_on_video/example/RoripwjYFp8_60.0_210.0.mp4"
        query_path = "run_on_video/example/queries.jsonl"
        queries = load_jsonl(query_path)
    query_text_list = [e["query"] for e in queries]
    ckpt_path = "run_on_video/CLIP_ckpt/qvhighlights_onlyCLIP/model_best.ckpt"

    # run predictions
    print("Build models...")
    clip_model_name_or_path = "ViT-B/32"
    # clip_model_name_or_path = "tmp/ViT-B-32.pt"
    cg_detr_predictor = CGDETRPredictor(
        ckpt_path=ckpt_path,
        clip_model_name_or_path=clip_model_name_or_path,
        device="cuda"
    )
    print("Run prediction...")
    predictions = cg_detr_predictor.localize_moment(
        video_path=video_path, query_list=query_text_list)

    # print data
    for idx, query_data in enumerate(queries):
        print("-"*30 + f"idx{idx}")
        print(f">> query: {query_data['query']}")
        print(f">> video_path: {video_path}")
        print(f">> Predicted moments ([start_in_seconds, end_in_seconds, score]): "
              f"{predictions[idx]['pred_relevant_windows']}")
        pred_saliency_scores = torch.Tensor(predictions[idx]['pred_saliency_scores'])
        bias = 0 - pred_saliency_scores.min()
        pred_saliency_scores += bias
        print(f">> Most saliency clip is (for all 2-sec clip): "
              f"{pred_saliency_scores.argmax()}")
        print(f">> Predicted saliency scores (for all 2-sec clip): "
              f"{pred_saliency_scores.tolist()}")
        if youtube_url == '':
            print(f">> GT moments: {query_data['relevant_windows']}")
            print(f">> GT saliency scores (only localized 2-sec clips): {query_data['saliency_scores']}")

In [24]:
if __name__ == "__main__":
    run_example()

Build models...
Loading feature extractors...
Loading CLIP models
Loading trained CG-DETR model...
Run prediction...
ffprobe failed at: run_on_video/example/RoripwjYFp8_60.0_210.0.mp4


TypeError: unsupported operand type(s) for /: 'dict' and 'float'